# Practical Project: USGS Earthquakes Pipeline

In this project we will build a small but real **scientific data pipeline** using the **USGS Earthquake Catalog API**.  

### What will be produced:
Artifacts (all under a project folder):
* `data/raw/`: raw API snapshots + metadata (query params, timestamps)
* `data/staged/`: parsed/normalized table (deduped, typed)
* `data/warehouse/`: curated table (Parquet; optionally partitioned by day)
* `data/reference/validation_report.json`: contracts + anomaly rates + canaries
* `data/reference/pipeline_runs/`: run logs for reproducibility

## 0 - Setup

Project will be created in the path relative to this notebook  

`/work/m2_project/`

In [1]:
from __future__ import annotations

from pathlib import Path
from datetime import datetime, timedelta, timezone
import json
import hashlib
import math

import numpy as np
import pandas as pd

from IPython.display import display

pd.set_option("display.max_columns", 160)
pd.set_option("display.width", 180)

WORK_DIR = Path("work")
PROJECT_DIR = WORK_DIR / "m2_project"

DATA_DIR = PROJECT_DIR / "data"
RAW_DIR = DATA_DIR / "raw"
STAGED_DIR = DATA_DIR / "staged"
WH_DIR = DATA_DIR / "warehouse"
REF_DIR = DATA_DIR / "reference"
RUN_DIR = REF_DIR / "pipeline_runs"

for p in [RAW_DIR, STAGED_DIR, WH_DIR, REF_DIR, RUN_DIR]:
    p.mkdir(parents=True, exist_ok=True)

print("Project:", PROJECT_DIR)
print("Raw:", RAW_DIR)
print("Staged:", STAGED_DIR)
print("Warehouse:", WH_DIR)
print("Reference:", REF_DIR)
print("Runs:", RUN_DIR)

Project: work/m2_project
Raw: work/m2_project/data/raw
Staged: work/m2_project/data/staged
Warehouse: work/m2_project/data/warehouse
Reference: work/m2_project/data/reference
Runs: work/m2_project/data/reference/pipeline_runs


### Helper Utilities  

These helpers keep the notebook focused on pipeline thinking rather than boilerplate

In [2]:
class PipelineError(RuntimeError):
    pass

def utc_now_iso() -> str:
    return datetime.now(timezone.utc).isoformat()

def sha16(x: str) -> str:
    return hashlib.sha256(x.encode("utf-8")).hexdigest()[:16]

def write_json(path: Path, obj: dict) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)
    path.write_text(json.dumps(obj, indent=2, default=str))

def read_json(path: Path) -> dict:
    return json.loads(path.read_text())

def require_columns(df: pd.DataFrame, cols: list[str], context: str) -> None:
    missing = [c for c in cols if c not in df.columns]
    if missing:
        raise PipelineError(f"[{context}] Missing required columns: {missing}")

def require_unique(df: pd.DataFrame, key: str, context: str) -> None:
    if key not in df.columns:
        raise PipelineError(f"[{context}] Missing key column '{key}'")
    dupes = int(df[key].duplicated().sum())
    if dupes:
        raise PipelineError(f"[{context}] Key '{key}' has {dupes} duplicates")

print("Helpers ready.")

Helpers ready.


## 1 - Ingest: Pull Earthquakes From USGS API (Paginated)

We will use the USGS **event** endpoint (GeoJSON):
* base `https://earthquak.usgs.gov/fdsnws/event/1/query`
* params `format=geojson`, `starttime`, `endtime`, `minmagnitude`, plus pagination (`limit, offset`)

### Choose a query window
Set:
* `DAYS_BACK` = 30
* `MIN_MAG` = 2.5

Then build `starttime` and `endtime` in **UTC** as ISO dates

**Note:** The API likes `YYYY-MM-DD` strings

In [3]:
# Set query window
DAYS_BACK = 30
MIN_MAG = 2.5

# compute starttime/endtime as YYYY-MM-DD (UTC-based)
endtime_dt = datetime.now(timezone.utc)
starttime_dt = endtime_dt - timedelta(days=DAYS_BACK)

starttime = starttime_dt.strftime('%Y-%m-%d')
endtime = endtime_dt.strftime('%Y-%m-%d')

print(f"starttime: {starttime}")
print(f"endtime: {endtime}")
print(f"minmagnitude: {MIN_MAG}")

starttime: 2026-01-04
endtime: 2026-02-03
minmagnitude: 2.5


### Implement pagination

Write `fetch_usgs_pages(...)` that:
* requests pages using `limit` and `offset`
* stops when a page returns fewer than `limit` features
* returns a list of page dictionaries

**Constraints:**  
* Use a small `limit` while testing (e.g., 200) to see pagination work.
* Add a polite `sleep` if wanted, but keep it simple

**Note:**
* `obj["features"]` = list of events
* `obj["metadata"]["count"]` = count for the query (not always equal to returned features)

In [4]:
import time
import requests

USGS_URL = "https://earthquake.usgs.gov/fdsnws/event/1/query"

def fetch_usgs_pages(starttime: str, endtime: str, minmag: float, limit: int=200, max_pages: int=50) -> list[dict]:
    pages = []
    offset = 1  # USGS uses 1-based offsets
    for page_i in range(max_pages):
        params = {
            "format": "geojson",
            "starttime": starttime,
            "endtime": endtime,
            "minmagnitude": minmag,
            "limit": limit,
            "offset": offset,
            "orderby": "time",
        }

        # Request the page, check the status, parse JSON
        print(f"Fetching page {page_i + 1} (offset={offset})...", end=" ")
        response = requests.get(USGS_URL, params=params)

        # Check for HTTP erros
        response.raise_for_status()

        data = response.json()
        features = data.get("features", [])
        total_count = data.get("metadata", {}).get("count", 0)
        print(f"got {len(features)} events (total in query: {total_count})")

        # Quick stop if there are no results
        if not features:
            print("No more results, stopping")
            break

        pages.append(data)

        # Stop if we get fewer results than the limit (last page)
        if len(features) < limit:
            print("Partial page received, stopping")
            break

        # Increment offset for next page
        offset += limit

        # Short polite break for API
        time.sleep(0.5)

    return pages

print("Fetcher ready.")
        

Fetcher ready.


### Fetch data and write a raw snapshot

Run the fetch and write:  
* raw pages: `data/raw/usgs_pages_<runid>.jsonl`
* raw_metadata: `data/raw/usgs_meta_<runid>.json`

**Note:**  
`.jsonl` means "JSON lines": one JSON object per line

In [7]:
from pprint import pprint

# Fetch data and write raw snapshot

# Compute run_id
RUN_ID = datetime.now(timezone.utc).strftime("%Y%m%d_%H%M%S_utc")

# Fetch pages
pages = fetch_usgs_pages(starttime, endtime, MIN_MAG, limit=200)
print(f"\nFetched {len(pages)} pages")

# Count total events
total_events = sum(len(p['features']) for p in pages)
print(f"Total events: {total_events}")

# Write raw pages as JSONL (one JSON object per line)
pages_path = RAW_DIR / f"usgs_pages_{RUN_ID}.jsonl"
pages_path.parent.mkdir(parents=True, exist_ok=True)

with pages_path.open('w') as f:
    for page in pages:
        f.write(json.dumps(page) + '\n')

print(f"Wrote pages: {pages_path}")

# Write metadata
metadata = {
    "run_id": RUN_ID,
    "generated_at_utc": utc_now_iso(),
    "query": {
        "starttime": starttime,
        "endtime": endtime,
        "minmagnitude": MIN_MAG,
    },
    "n_pages": len(pages),
    "n_features_total": total_events,
    "source": "USGS Earthquake Catalog (GeoJSON)",
    "endpoint": USGS_URL,
}

meta_path = RAW_DIR / f"usgs_meta_{RUN_ID}.json"
write_json(meta_path, metadata)
print(f"Wrote metadata: {meta_path}")


# Pretty print the metadata
print("\nMetadata:")
pprint(metadata, sort_dicts=False)
      

Fetching page 1 (offset=1)... got 200 events (total in query: 200)
Fetching page 2 (offset=201)... got 200 events (total in query: 200)
Fetching page 3 (offset=401)... got 200 events (total in query: 200)
Fetching page 4 (offset=601)... got 200 events (total in query: 200)
Fetching page 5 (offset=801)... got 200 events (total in query: 200)
Fetching page 6 (offset=1001)... got 200 events (total in query: 200)
Fetching page 7 (offset=1201)... got 200 events (total in query: 200)
Fetching page 8 (offset=1401)... got 190 events (total in query: 190)
Partial page received, stopping

Fetched 8 pages
Total events: 1590
Wrote pages: work/m2_project/data/raw/usgs_pages_20260203_171853_utc.jsonl
Wrote metadata: work/m2_project/data/raw/usgs_meta_20260203_171853_utc.json

Metadata:
{'run_id': '20260203_171853_utc',
 'generated_at_utc': '2026-02-03T17:18:58.340696+00:00',
 'query': {'starttime': '2026-01-04',
           'endtime': '2026-02-03',
           'minmagnitude': 2.5},
 'n_pages': 8,
 'n_

## 2 - Stage: Normalize GeoJSON &rarr; Table

USGS GeoJSON structure:
* `feature["id"]` is a stable event id
* `feature["properties"]` contains magnitude, place, time, etc.
* `feature["geometry"]["coordinates"]`is`[longitude, latitude, depth_km]`

### Flatten features into a DataFrame  

Implement `features_to_df(pages)` that returns one DataFrame with one row per event.

**Note:**  
`pd.json_normalize(features)` will help

In [8]:
# Create a DataFrame with one row per event
def features_to_df(pages: list[dict]) -> pd.DataFrame:
    # 1. Extract all features from all pages
    all_features = []
    for page in pages:
        all_features.extend(page.get("features", []))

    # 2. Use pd.json_normalize to flatten nested structures, automatically
    df = pd.json_normalize(all_features)

    # 3. Extract coordinates into separate columns
    # geometry.coordintates is [longitude, latitude, depth km]
    df['longitude'] = df['geometry.coordinates'].apply(lambda x: x[0] if x else None)
    df['latitude'] = df['geometry.coordinates'].apply(lambda x: x[1] if x else None)
    df['depth_km'] = df['geometry.coordinates'].apply(lambda x: x[2] if x else None)

    # 4. Clean up column names (remove properties prefix)
    df.columns = df.columns.str.replace('properties.', '', regex=False)

    # 5. Drop the original nested column (geometry.coordinates)
    df = df.drop(columns=['geometry.coordinates'], errors='ignore')

    return df

print("flattening function ready")

flattening function ready


In [9]:
df_raw = features_to_df(pages)
print(f"Raw flattened shape: {df_raw.shape}")
display(df_raw.head(5))

Raw flattened shape: (1590, 32)


,type,id,mag,place,time,updated,tz,url,detail,felt,cdi,mmi,alert,status,tsunami,sig,net,code,ids,sources,types,nst,dmin,rms,gap,magType,type,title,geometry.type,longitude,latitude,depth_km
0,Feature,us6000s620,4.10,"16 km E of Calingasta, Argentina",1770075292434,1770080742040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,None,reviewed,0,259,us,6000s620,",us6000s620,",",us,",",origin,phase-data,",27,1.3220,0.79,89.0,mb,earthquake,"M 4.1 - 16 km E of Calingasta, Argentina",Point,-69.251700,-31.317600,162.179
1,Feature,uw62216847,2.80,"5 km ESE of Benton City, Washington",1770075173110,1770096410010,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,None,reviewed,0,121,uw,62216847,",uw62216847,",",uw,",",origin,phase-data,",14,0.1610,0.28,202.0,ml,explosion,"M 2.8 Explosion - 5 km ESE of Benton City, Was...",Point,-119.422000,46.234500,-0.240
2,Feature,nc75306276,2.94,"5 km SE of San Ramon, CA",1770073598570,1770101775717,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,10.0,3.3,NaN,None,automatic,0,136,nc,75306276,",nc75306276,us6000s61u,",",nc,us,",",dyfi,focal-mechanism,nearby-cities,origin,pha...",64,0.1036,0.10,37.0,ml,earthquake,"M 2.9 - 5 km SE of San Ramon, CA",Point,-121.935837,37.754501,7.370
3,Feature,nc75306271,3.00,"4 km ESE of San Ramon, CA",1770073458430,1770134850183,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,25.0,4.1,NaN,None,reviewed,0,149,nc,75306271,",nc75306271,us6000s61t,",",nc,us,",",dyfi,focal-mechanism,nearby-cities,origin,pha...",74,0.1056,0.10,39.0,ml,earthquake,"M 3.0 - 4 km ESE of San Ramon, CA",Point,-121.935333,37.763332,8.380
4,Feature,nc75306256,3.14,"4 km ESE of San Ramon, CA",1770072973340,1770098210597,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,97.0,3.4,NaN,None,reviewed,0,185,nc,75306256,",nc75306256,us6000s61s,",",nc,us,",",dyfi,focal-mechanism,nearby-cities,origin,pha...",80,0.1033,0.12,26.0,ml,earthquake,"M 3.1 - 4 km ESE of San Ramon, CA",Point,-121.937164,37.765167,7.820


### Parse types and extract coordinates  

Create a staged table with columns:  
* `event_id`: string
* `time_utc`: datetime
* `updated_utc`: datetime
* `mag`: float
* `place`: string
* `longitude`: float
* `latitude`: float
* `depth_km`: float
* `tsunami`: int
* `status`: string

USGS times are often **milliseconds since epoch**  

**Note:** `pd.to_datetime(ms, unit="ms", utc=True)  

Also:
* Deduplicate by `event_id` (keep first)
* Normalize missing-like strings

In [10]:
# Build staged dataframe
def staged_dataframe(data: pd.DataFrame) -> pd.DataFrame:
    MISSING_TOKENS = {"", "nan", "None", "UNKNOWN", "Unknown", "N/A", "NA", "NULL", "null"}
    
    staged = pd.DataFrame()

    # 1. event_id: basic identification - rename and ensure string type
    staged['event_id'] = data['id'].astype(str)

    # 2. time_utc: parse timestamps from unix milliseconds to datetime
    staged['time_utc'] = pd.to_datetime(data['time'], unit='ms', utc=True)

    # 3. updated_utc: parse timestamps from unix milliseconds to datetime
    staged['updated_utc'] = pd.to_datetime(data['updated'], unit='ms', utc=True)

    # 4. mag: handle missing values and convert to float (coerce = NaN)
    staged['mag'] = pd.to_numeric(data['mag'], errors='coerce')

    # 5. place: location description - ensure string type
    staged['place'] = data['place'].astype(str)

    # 6. coordinates (longitude, latitude, depth_km): already extracted, just add and handle missing values
    staged['longitude'] = pd.to_numeric(data['longitude'], errors='coerce')
    staged['latitude'] = pd.to_numeric(data['latitude'], errors='coerce')
    staged['depth_km'] = pd.to_numeric(data['depth_km'], errors='coerce')

    # 7. tsunami: flag to indicate a tsunami - convert to int, fill missing with 0
    staged['tsunami'] = pd.to_numeric(data['tsunami'], errors='coerce').fillna(0).astype(int)

    # 8. status: review status - convert to string
    staged['status'] = data['status'].astype(str)

    # 9. Normalize missing-like strings to NaN for all string columns
    for col in staged.columns:
        if staged[col].dtype == 'object':  # string columns have dtype 'object'
            staged[col] = staged[col].replace(MISSING_TOKENS, np.nan)

    # 10. Delete duplicate entries by event_id. Keep the first entry
    entries_prior = len(staged)
    staged = staged.sort_values('time_utc', ascending=False)
    staged = staged.drop_duplicates(subset=['event_id'], keep='first')
    entries_after = len(staged)
    print(f"Deduped: {entries_prior} -> {entries_after}")

    return staged

print("function to create staged DataFrame ready")

function to create staged DataFrame ready


In [11]:
df_staged = staged_dataframe(df_raw)
print(f"Staged shape: {df_staged.shape}")
display(df_staged.head(5))

Deduped: 1590 -> 1590
Staged shape: (1590, 10)


,event_id,time_utc,updated_utc,mag,place,longitude,latitude,depth_km,tsunami,status
0,us6000s620,2026-02-02 23:34:52.434000+00:00,2026-02-03 01:05:42.040000+00:00,4.10,"16 km E of Calingasta, Argentina",-69.251700,-31.317600,162.179,0,reviewed
1,uw62216847,2026-02-02 23:32:53.110000+00:00,2026-02-03 05:26:50.010000+00:00,2.80,"5 km ESE of Benton City, Washington",-119.422000,46.234500,-0.240,0,reviewed
2,nc75306276,2026-02-02 23:06:38.570000+00:00,2026-02-03 06:56:15.717000+00:00,2.94,"5 km SE of San Ramon, CA",-121.935837,37.754501,7.370,0,automatic
3,nc75306271,2026-02-02 23:04:18.430000+00:00,2026-02-03 16:07:30.183000+00:00,3.00,"4 km ESE of San Ramon, CA",-121.935333,37.763332,8.380,0,reviewed
4,nc75306256,2026-02-02 22:56:13.340000+00:00,2026-02-03 05:56:50.597000+00:00,3.14,"4 km ESE of San Ramon, CA",-121.937164,37.765167,7.820,0,reviewed


### Write staged output  

Write:  
* `data/staged/usgs_earthquakes_staged_<runid>.parquet`
* `data/staged/usgs_staged_meta_<runid>.json`

In [51]:
# Determine file paths
staged_parquet_path = STAGED_DIR / f"usgs_earthquakes_staged_{RUN_ID}.parquet"
staged_metadata_path = STAGED_DIR / f"usgs_staged_meta_{RUN_ID}.json"

# Create staged metadata
staged_metadata = {
    "run_id": RUN_ID,
    "generated_at_utc": datetime.now(timezone.utc).isoformat(),
    "pipeline_stage": "staged",

    "input": {
        "source": "USGS Earthquake API",
        "api_endpoint": USGS_URL,
        "raw_records_fetched": len(df_raw),
    },

    "output": {
        "parquet_file": staged_parquet_path.name,
        "parquet_path": str(staged_parquet_path),
        "rows": len(df_staged),
        "columns": len(df_staged.columns),
        "column_list": df_staged.columns.tolist(),
    },

    "schema": {
        col: str(dtype) for col, dtype in df_staged.dtypes.items()
    },
    
}

# Write outputs
df_staged.to_parquet(staged_parquet_path, index=False)
print(f"Wrote: {staged_parquet_path}")
with open(staged_metadata_path, 'w') as f:
    json.dump(staged_metadata, f, indent=2)
print(f"Wrote: {staged_metadata_path}")


Wrote: work/m2_project/data/staged/usgs_earthquakes_staged_20260203_171853_utc.parquet
Wrote: work/m2_project/data/staged/usgs_staged_meta_20260203_171853_utc.json


## 3 - Curate: Build Analysis-Ready Features

Create a curated table with:  
* time features: `event_day`, `event_hour`, `dayofweek`, `is_weekend`
* magnitude bins and flags
* a lightweight region extraction from place

### Feature Construction  

Implement:
* `event_day`: date(UTC)
* `event_hour`: hour(UTC)
* `mag_bin`: (example bins): <2.5, 2.5-4.5, 4.5-6.0, 6.0+
* `is_major`: mag >= 6.0
* `region`: parsed from `place`:
   * if `place` contains `" of "`, take the substring after the last `" of "`
   * otherwise use the full `place`

**Note:**
`str.rsplit(" of", 1)`

In [13]:
def parse_region(place: str) -> str:
    if pd.isna(place):
        return None

    if " of " in place:
        return place.rsplit(" of ", 1)[1]
    else:
        return place

print("Region parser function ready")

Region parser function ready


In [16]:
def curated_dataframe(data: pd.DataFrame) -> pd.DataFrame:
    # Build a curated DataFrame
    df_curated = data.copy()
    
    # 1. Extract date (removes time component)
    df_curated['event_day'] = df_curated['time_utc'].dt.date
    
    # 2. Extract hour
    df_curated['event_hour'] = df_curated['time_utc'].dt.hour
    
    # 3. Extract day of week
    df_curated['dayofweek'] = df_curated['time_utc'].dt.dayofweek
    
    # 4. Weekend flag
    df_curated['is_weekend'] = df_curated['dayofweek'].isin([5, 6]).astype(int)
    
    # 5. Magnitude bins
    # define
    bins = [-float('inf'), 2.5, 4.5, 6.0, float('inf')]
    labels = ['<2.5', '2.5-4.5', '4.5-6.0', '6.0+']
    # add
    df_curated['mag_bin'] = pd.cut(df_curated['mag'], bins=bins, labels=labels)
    
    # 6. Flag for major earthquakes (>6.0)
    df_curated['is_major'] = (df_curated['mag'] >- 6.0).astype(int)
    
    # 7. Parse region
    df_curated['region'] = df_curated['place'].apply(parse_region)

    return df_curated

print("curated dataframe function ready")

curated dataframe function ready


In [17]:
df_curated = curated_dataframe(df_staged)
print(f"Curated shape: {df_curated.shape}")
display(df_curated.head(5))

Curated shape: (1590, 17)


,event_id,time_utc,updated_utc,mag,place,longitude,latitude,depth_km,tsunami,status,event_day,event_hour,dayofweek,is_weekend,mag_bin,is_major,region
0,us6000s620,2026-02-02 23:34:52.434000+00:00,2026-02-03 01:05:42.040000+00:00,4.10,"16 km E of Calingasta, Argentina",-69.251700,-31.317600,162.179,0,reviewed,2026-02-02,23,0,0,2.5-4.5,1,"Calingasta, Argentina"
1,uw62216847,2026-02-02 23:32:53.110000+00:00,2026-02-03 05:26:50.010000+00:00,2.80,"5 km ESE of Benton City, Washington",-119.422000,46.234500,-0.240,0,reviewed,2026-02-02,23,0,0,2.5-4.5,1,"Benton City, Washington"
2,nc75306276,2026-02-02 23:06:38.570000+00:00,2026-02-03 06:56:15.717000+00:00,2.94,"5 km SE of San Ramon, CA",-121.935837,37.754501,7.370,0,automatic,2026-02-02,23,0,0,2.5-4.5,1,"San Ramon, CA"
3,nc75306271,2026-02-02 23:04:18.430000+00:00,2026-02-03 16:07:30.183000+00:00,3.00,"4 km ESE of San Ramon, CA",-121.935333,37.763332,8.380,0,reviewed,2026-02-02,23,0,0,2.5-4.5,1,"San Ramon, CA"
4,nc75306256,2026-02-02 22:56:13.340000+00:00,2026-02-03 05:56:50.597000+00:00,3.14,"4 km ESE of San Ramon, CA",-121.937164,37.765167,7.820,0,reviewed,2026-02-02,22,0,0,2.5-4.5,1,"San Ramon, CA"


### Choose curated columns and write Parquet  

Write:  
* `data/warehouse/usgs_earthquakes_curated_<runid>.parquet`
* partition by `event_day` under `data/warehouse/partitions/event_day=YYYY-MM-DD/earthquakes.parquet`


In [24]:
# Select curated columns
curated_columns = [
    # Identification
    "event_id",

    # Time features
    "time_utc",
    "event_day",
    "event_hour",
    "dayofweek",
    "is_weekend",

    # Location
    "longitude",
    "latitude",
    "depth_km",
    "place",
    "region",

    # Magnitude
    "mag",
    "mag_bin",
    "is_major",

    # Impact/flags
    "tsunami",
    "status",
]

# Create final curated datafram with selected columns and write
df_curated_final = df_curated[curated_columns].copy()
curated_parquet_path = WH_DIR / f"usgs_earthquakes_curated_{RUN_ID}.parquet"
df_curated_final.to_parquet(curated_parquet_path, index=False)
print(f"Wrote curated parquet: {curated_parquet_path.name}")

# Write partitioned parquet
PART_DIR = WH_DIR / "partitions"
PART_DIR.mkdir(parents=True, exist_ok=True)
partition_count = 0
for day, day_data in df_curated_final.groupby('event_day'):
    day_str = str(day)
    partition_dir = PART_DIR / f"event_day={day_str}"
    partition_dir.mkdir(parents=True, exist_ok=True)

    partition_path = partition_dir / "earthquakes.parquet"
    day_data.to_parquet(partition_path, index=False)
    partition_count += 1

print(f"Wrote {partition_count} partitions to {PART_DIR.relative_to(WORK_DIR)}")



Wrote curated parquet: usgs_earthquakes_curated_20260203_171853_utc.parquet
Wrote 30 partitions to m2_project/data/warehouse/partitions


## 4 - Validate: Contracts + Anomalies + Canaries

### Define Contracts

In [26]:
# Determine required columns
REQUIRED_COLS = {
    "event_id",
    "time_utc",
    "mag",
    "latitude",
    "longitude",
    "depth_km",
}

# Determine valid ranges
MAG_MIN = -1.0
MAG_MAX = 10.0
DEPTH_MIN = -10.0
DEPTH_MAX = 700.0

print("Contracts defined:")
print(f"  Required cols: {REQUIRED_COLS}")
print(f"  Magnitude range: ({MAG_MIN}, {MAG_MAX})")
print(f"  Depth range (km): ({DEPTH_MIN}, {DEPTH_MAX})")

Contracts defined:
  Required cols: {'longitude', 'time_utc', 'depth_km', 'mag', 'event_id', 'latitude'}
  Magnitude range: (-1.0, 10.0)
  Depth range (km): (-10.0, 700.0)


### Implement validation checks

In [31]:
def validate_contracts(data: pd.DataFrame) -> dict:
    results = {"passed": True, "checks": []}

    def record_check(name: str, passed: bool, details: str):
        results["checks"].append({
            "name": name,
            "passed": passed,
            "details": details
        })
        if not passed:
            results["passed"] = False

    # 1. Required columns exist
    missing_cols = set(REQUIRED_COLS) - set(data.columns)
    record_check(
        "required_columns",
        len(missing_cols) == 0,
        f"Missing columns: {list(missing_cols)}" if missing_cols else "All required columns present"
    )

    # 2. Non-empty dataset
    record_check(
        "non_empty",
        len(data) > 0,
        f"Dataset has {len(data)} rows"
    )

    # 3. event_id unique
    duplicate_count = data['event_id'].duplicated().sum()
    record_check(
        "event_id_unique",
        duplicate_count == 0,
        f"Found {duplicate_count} duplicate event_ids" if duplicate_count > 0 else "All event_ids unique"
    )

    # 4. Latitude range
    invalid_lat = (~data['latitude'].between(-90, 90)).sum()
    record_check(
        "latitude_range",
        invalid_lat == 0,
        f"Found {invalid_lat} invalid latitudes" if invalid_lat > 0 else "All latitudes in valid range [-90, 90]"
    )

    # 5. Longitude range
    invalid_lon = (~data['longitude'].between(-180, 180)).sum()
    record_check(
        "longitude_range",
        invalid_lon == 0,
        f"Found {invalid_lon} invalid longitudes" if invalid_lon > 0 else "All longitudes in valid range [-180, 180]"
    )

    # 6. Magnitude range
    invalid_mag = (~data['mag'].between(MAG_MIN, MAG_MAX)).sum()
    record_check(
        "magnitude_range",
        invalid_mag == 0,
        f"Found {invalid_mag} magnitudes outside [{MAG_MIN}, {MAG_MAX}]" if invalid_mag > 0 else f"All magnitudes in valid range [{MAG_MIN}, {MAG_MAX}]"
    )

    # 7. Depth Range
    invalid_depth = (~data['depth_km'].between(DEPTH_MIN, DEPTH_MAX)).sum()
    record_check(
        "depth_range",
        invalid_depth == 0,
        f"Found {invalid_depth} depths outside [{DEPTH_MIN}, {DEPTH_MAX}] km" if invalid_depth > 0 else f"All depths in valid range [{DEPTH_MIN}, {DEPTH_MAX}] km"
    )

    # 8. No future events
    now = pd.Timestamp.now(tz='UTC')
    future_events = (data['time_utc'] > now).sum()
    record_check(
        "no_future_events",
        future_events == 0,
        f"Found {future_events} events in the future" if future_events > 0 else "No future events detected"
    )

    return results

print("Validation function ready")

Validation function ready


In [33]:
validation_results = validate_contracts(df_curated_final)

failures = sum(1 for check in validation_results["checks"] if not check["passed"])

print(f"\nValidation passed? {validation_results['passed']}")
print(f"Failures: {failures}")


Validation passed? True
Failures: 0


### Anomaly flags and investigation table

In [34]:
# Create anomaly dataframe
anomalies = pd.DataFrame(index=df_curated_final.index)

# 1. Missing magnitude
anomalies['anom_mag_missing'] = df_curated_final['mag'].isna().astype(int)

# 2. Extreme magnitude
# Flag magnitudes outside typical range (< 0 or > 8.0)
anomalies['anom_mag_extreme'] = (
    (df_curated_final['mag'] < 0) | 
    (df_curated_final['mag'] > 8.0)
).astype(int)

# 3. Negative depth
# Events above sea level (unusual but possible)
anomalies['anom_depth_negative'] = (df_curated_final['depth_km'] < 0).astype(int)

# 4. Extreme depth
# Very deep events (> 500 km)
anomalies['anom_depth_extreme'] = (df_curated_final['depth_km'] > 500).astype(int)

# 5. Missing coordinates
anomalies['anom_coords_missing'] = (
    df_curated_final['longitude'].isna() | 
    df_curated_final['latitude'].isna()
).astype(int)

# Summary - Any anomaly
anomalies['anom_any'] = (anomalies.sum(axis=1) > 0).astype(int)

# Count suspicious rows
suspicious_count = anomalies['anom_any'].sum()
print(f"Suspicious rows: {suspicious_count}")

# Create investigation table
if suspicious_count > 0:
    # Select columns for investigation
    investigation_cols = [
        'event_id', 'time_utc', 'mag', 'depth_km', 
        'latitude', 'longitude', 'region', 'place'
    ]
    
    # Combine curated data with anomaly flags
    investigation = pd.concat([
        df_curated_final[investigation_cols],
        anomalies
    ], axis=1)
    
    # Filter to only suspicious rows
    investigation = investigation[investigation['anom_any'] == 1]
    
    # Display investigation table
    display(investigation)
else:
    print("No anomalies detected")

Suspicious rows: 38


,event_id,time_utc,mag,depth_km,latitude,longitude,region,place,anom_mag_missing,anom_mag_extreme,anom_depth_negative,anom_depth_extreme,anom_coords_missing,anom_any
1,uw62216847,2026-02-02 23:32:53.110000+00:00,2.80,-0.240,46.234500,-119.422000,"Benton City, Washington","5 km ESE of Benton City, Washington",0,0,1,0,0,1
56,us6000s5vj,2026-02-01 18:09:34.279000+00:00,4.70,527.156,-23.597700,179.933700,the Fiji Islands,south of the Fiji Islands,0,0,0,1,0,1
83,us6000s5s2,2026-01-31 20:08:33.970000+00:00,4.40,507.818,-24.978100,179.786200,the Fiji Islands,south of the Fiji Islands,0,0,0,1,0,1
87,us6000s5rp,2026-01-31 18:34:43.926000+00:00,4.80,584.727,6.738000,123.626900,"Gadung, Philippines","44 km W of Gadung, Philippines",0,0,0,1,0,1
91,us6000s5rc,2026-01-31 15:59:11.987000+00:00,4.40,530.235,-20.003200,-177.552700,"Houma, Tonga","267 km WNW of Houma, Tonga",0,0,0,1,0,1
175,us6000s5bj,2026-01-29 20:13:42.056000+00:00,4.70,544.114,-20.525300,-178.214000,Fiji region,Fiji region,0,0,0,1,0,1
282,uw62216317,2026-01-27 22:26:04.650000+00:00,2.61,-0.430,42.653833,-124.439167,"Port Orford, Oregon","11 km SSE of Port Orford, Oregon",0,0,1,0,0,1
283,uw62216302,2026-01-27 21:25:04.700000+00:00,3.05,-0.640,42.644500,-124.445667,"Port Orford, Oregon","12 km SSE of Port Orford, Oregon",0,0,1,0,0,1
306,us6000s4p9,2026-01-27 08:31:31.478000+00:00,4.50,573.799,-4.367600,149.686000,"Kimbe, Papua New Guinea","140 km NNW of Kimbe, Papua New Guinea",0,0,0,1,0,1
309,us6000s4p3,2026-01-27 07:54:37.140000+00:00,4.20,572.586,-23.433300,178.866100,the Fiji Islands,south of the Fiji Islands,0,0,0,1,0,1


### Canaries + spike days

In [39]:
# Compute daily event counts
daily_counts = df_curated_final.groupby('event_day').size().reset_index(name='n_events')

# Canary metrics
canaries = {
    "n_days": len(daily_counts),
    "min": int(daily_counts["n_events"].min()),
    "median": float(daily_counts["n_events"].median()),
    "max": int(daily_counts["n_events"].max())
}

# Spike detection
median_count = canaries['median']
spike_threshold = 2.5 * median_count

# find spike days
spike_days = daily_counts[daily_counts["n_events"] > spike_threshold]["event_day"].tolist()

print((canaries, spike_days))

({'n_days': 30, 'min': 24, 'median': 50.0, 'max': 90}, [])


## 5 - Leakage Audit

### Leakage Audit Checklist

### Understanding Each Check:

#### 1. "Any preprocessing stats (bins, encoders) computed on TRAIN only?"
* **Bad**: Compute magnitude bins using min/max from entire dataset
* **Good**: Compute bins only on training set, apply to test set
* **Why**: Test set info shouldn't influence training preprocessing

#### 2. "Any joins pulling in data that arrives after event time?"
* **Bad**: Join earthquake with aftershock counts (aftershocks happen AFTER)
* **Good**: Only use data available at time of earthquake
* **Why**: At prediction time, you won't have future data

#### 3. "Are you using updated_utc as a feature when it may not exist at prediction time?"
* **Bad**: Use `updated_utc` as a feature (it gets revised later)
* **Good**: Only use `time_utc` (the event occurrence time)
* **Why**: `updated_utc` reflects when USGS last revised the data, not event properties

#### 4. "Are you computing group summaries using future rows (time leakage)?"
* **Bad**: "Average magnitude in this region this month" (includes future events in same month)
* **Good**: "Average magnitude in this region in past 30 days" (only uses past)
* **Why**: You can't know future events when making predictions

#### 5. "Is the prediction time clearly defined for the task?"
* **Important**: When exactly are you making the prediction?
   * At earthquake detection time? (few minutes after)
   * 1 hour after earthquake?
   * Next day?
* **Why**: Defines what data is "available" vs "future"

In [41]:
leakage_checklist = [
    "Any preprocessing stats (bins, encoders) computed on TRAIN only?",
    "Any joins pulling in data that arrives after event time?",
    "Are you using updated_utc as a feature when it may not exist at prediction time?",
    "Are you computing group summaries using future rows (time leakage)?",
    "Is the prediction time clearly defined for the task?",
]

display(leakage_checklist)

['Any preprocessing stats (bins, encoders) computed on TRAIN only?',
 'Any joins pulling in data that arrives after event time?',
 'Are you using updated_utc as a feature when it may not exist at prediction time?',
 'Are you computing group summaries using future rows (time leakage)?',
 'Is the prediction time clearly defined for the task?']

## 6 - Write a Validation Report + Run Log

In [44]:
# Build Validation Report
validation_report = {
    "run_id": RUN_ID,
    "generated_at_utc": datetime.now(timezone.utc).isoformat(),
    "dataset": "USGS Earthquake Data - Curated",
    "n_rows": int(len(df_curated_final)),
    "n_cols": int(len(df_curated_final.columns)),
    
    # Contract validation results
    "contracts": {
        "defined": {
            "required_cols": list(REQUIRED_COLS),
            "mag_range": [float(MAG_MIN), float(MAG_MAX)],
            "depth_range": [float(DEPTH_MIN), float(DEPTH_MAX)],
        },
        "validation_passed": bool(validation_results["passed"]),
        "failures": int(sum(1 for check in validation_results["checks"] if not check["passed"])),
        "checks": [
            {
                "name": check["name"],
                "passed": bool(check["passed"]),
                "details": str(check["details"])
            }
            for check in validation_results["checks"]
        ],
    },
    # Anomaly detection results
    "anomalies": {
        "suspicious_rows": int(anomalies['anom_any'].sum()),
        "suspicious_pct": float(anomalies['anom_any'].mean() * 100),
        "by_type": {
            col: int(anomalies[col].sum()) 
            for col in anomalies.columns if col.startswith('anom_') and col != 'anom_any'
        },
    },
    # Canary metrics
    "canaries": {
        "daily_events": canaries,
        "spike_days": [str(d) for d in spike_days],
        "spike_threshold": float(spike_threshold),
    },
    # Leakage audit checklist
    "leakage_checklist": leakage_checklist,
}

# Write Validation Report
validation_report_path = REF_DIR / f"validation_report_{RUN_ID}.json"
with open(validation_report_path, 'w') as f:
    json.dump(validation_report, f, indent=2)

print(f"Wrote validation report: {validation_report_path.name}")

Wrote validation report: validation_report_20260203_171853_utc.json


### Write a run log

In [52]:
# Pipeline Run Log
run_log = {
    "run_id": f"run_{RUN_ID}",
    "generated_at_utc": datetime.now(timezone.utc).isoformat(),
    
    # Input tracking
    "inputs": {
        "query_fingerprint": hashlib.md5(
            f"{starttime}{endtime}{MIN_MAG}".encode()
        ).hexdigest()[:16],
        "raw_pages_path": str((RAW_DIR / f"usgs_pages_run_{RUN_ID}.jsonl")),
        "raw_pages_size_bytes": (RAW_DIR / f"usgs_pages_run_{RUN_ID}.jsonl").stat().st_size if (RAW_DIR / f"usgs_pages_run_{RUN_ID}.jsonl").exists() else 0,
        "raw_meta_path": str(RAW_DIR / f"usgs_meta_run_{RUN_ID}.json"),
    },
    
    # Output tracking
    "outputs": {
        "staged_path": str(staged_parquet_path),
        "curated_path": str(curated_parquet_path),
        "partition_root": str(PART_DIR),
        "validation_report_path": str(validation_report_path),
    },
    
    # Row definition
    "row_definition": "Each row is one earthquake event from USGS (event_id) at time_utc.",
    
    # Pipeline notes
    "notes": [
        "If validations fail, do not silently continue. Investigate upstream changes or parsing issues.",
        "Consider adding alerts for spike days or missingness threshold breaches.",
    ],
}

# Write Run Log
run_log_path = RUN_DIR / f"run_{RUN_ID}.json"
with open(run_log_path, 'w') as f:
    json.dump(run_log, f, indent=2)

print(f"Saved: {run_log_path}")
print()
display(run_log)

Saved: work/m2_project/data/reference/pipeline_runs/run_20260203_171853_utc.json



{'run_id': 'run_20260203_171853_utc',
 'generated_at_utc': '2026-02-03T20:46:13.746210+00:00',
 'inputs': {'query_fingerprint': 'ae99c964a6922b52',
  'raw_pages_path': 'work/m2_project/data/raw/usgs_pages_run_20260203_171853_utc.jsonl',
  'raw_pages_size_bytes': 0,
  'raw_meta_path': 'work/m2_project/data/raw/usgs_meta_run_20260203_171853_utc.json'},
 'outputs': {'staged_path': 'work/m2_project/data/staged/usgs_earthquakes_staged_20260203_171853_utc.parquet',
  'curated_path': 'work/m2_project/data/warehouse/usgs_earthquakes_curated_20260203_171853_utc.parquet',
  'partition_root': 'work/m2_project/data/warehouse/partitions',
  'validation_report_path': 'work/m2_project/data/reference/validation_report_20260203_171853_utc.json'},
 'row_definition': 'Each row is one earthquake event from USGS (event_id) at time_utc.',
 'notes': ['If validations fail, do not silently continue. Investigate upstream changes or parsing issues.',
  'Consider adding alerts for spike days or missingness thresh

# USGS Earthquake Data Pipeline - Summary

## Overview
This notebook demonstrates a complete, production-ready data pipeline for earthquake data from the USGS API. It follows best practices for data engineering and ML pipeline design, structured around the **Extract → Stage → Curate → Validate** pattern.

---

## Pipeline Stages

### **Stage 0: Setup**
- Establishes workspace structure (`data/raw/`, `data/staged/`, `data/warehouse/`, `data/reference/`)
- Defines helper utilities for consistent file handling
- Generates a **run ID** (`RUN_ID`) that links all artifacts from a single execution
- Sets up pandas display options

**Key Learning:** Workspace organization and run tracking are foundational to reproducible pipelines.

---

### **Stage 1: Ingest - Pull Data from USGS API**
**Objective:** Fetch raw earthquake data with proper pagination and error handling

**What it does:**
- Queries USGS Earthquake API with configurable parameters (time window, minimum magnitude)
- Implements pagination to handle large result sets (200 events per page)
- Saves raw JSON responses to preserve original data structure
- Creates metadata tracking query parameters and fetch statistics

**Key Files Produced:**
- `data/raw/usgs_pages_run_{RUN_ID}.jsonl` - Raw API responses
- `data/raw/usgs_meta_run_{RUN_ID}.json` - Query metadata

**Best Practices:**
- ✅ Always save raw data exactly as received (no transformations yet)
- ✅ Track provenance: what query produced this data?
- ✅ Handle pagination and rate limiting
- ✅ Use consistent run IDs across all artifacts

---

### **Stage 2: Stage - Normalize GeoJSON → Table**
**Objective:** Transform nested JSON into typed, cleaned tabular data

**What it does:**
1. **Flatten nested structures** - Converts GeoJSON features to flat columns using `pd.json_normalize()`
2. **Extract coordinates** - Pulls longitude, latitude, depth from nested `geometry.coordinates` array
3. **Parse timestamps** - Converts Unix milliseconds to proper datetime objects with UTC timezone
4. **Type enforcement** - Converts strings to appropriate types (float, int, datetime)
5. **Normalize missing values** - Replaces sentinel values (`"None"`, `"NA"`, `""`) with proper `NaN`
6. **Deduplication** - Keeps most recent version of each event by `event_id`

**Key Files Produced:**
- `data/staged/usgs_earthquakes_staged_{RUN_ID}.parquet` - Typed, deduplicated data
- `data/staged/usgs_staged_meta_{RUN_ID}.json` - Schema and quality metadata

**Best Practices:**
- ✅ Staging focuses on structure/types, NOT feature engineering
- ✅ Use `errors='coerce'` for safe type conversions
- ✅ Explicit deduplication policy (keep most recent update)
- ✅ Document schema in metadata

**Key Function:** `staged_dataframe(df_raw)` - Encapsulates all staging logic

---

### **Stage 3: Curate - Build Analysis-Ready Features**
**Objective:** Create derived features for downstream analysis/modeling

**What it does:**
1. **Temporal features**
   - `event_day` - Date portion (for partitioning)
   - `event_hour` - Hour of day (0-23)
   - `dayofweek` - Day of week (0=Monday, 6=Sunday)
   - `is_weekend` - Binary flag for Saturday/Sunday

2. **Magnitude features**
   - `mag_bin` - Categorized magnitude (<2.5, 2.5-4.5, 4.5-6.0, 6.0+)
   - `is_major` - Binary flag for magnitude ≥ 6.0

3. **Location features**
   - `region` - Parsed from `place` string (e.g., "16 km E of San Ramon, CA" → "San Ramon, CA")

**Key Files Produced:**
- `data/warehouse/usgs_earthquakes_curated_{RUN_ID}.parquet` - Single curated file
- `data/warehouse/partitions/event_day={YYYY-MM-DD}/earthquakes.parquet` - Partitioned by day

**Best Practices:**
- ✅ Curated layer adds value (derived features), staged layer ensures quality (types/dedup)
- ✅ Partition by frequently-filtered columns (date) for query efficiency
- ✅ Select only relevant columns for curated output (drop technical metadata)
- ✅ Hive-style partitioning (`event_day=YYYY-MM-DD/`) for compatibility with Spark/Dask

**Why Partition?**
- Faster queries: "Show February 1st" reads 1 file, not 1000
- Incremental updates: Add new days without rewriting history
- Standard practice: Works with modern data tools (DuckDB, Polars, Spark)

---

### **Stage 4: Validate - Contracts + Anomalies + Canaries**
**Objective:** Ensure data quality and detect drift

#### **4.1 Contracts (Hard Requirements)**
Define and enforce expectations:
- **Required columns** - Must have `event_id`, `time_utc`, `mag`, `latitude`, `longitude`, `depth_km`
- **Valid ranges** - Latitude [-90, 90], Longitude [-180, 180], Magnitude [-1, 10], Depth [-10, 700]
- **Uniqueness** - `event_id` must be unique
- **Temporal validity** - No future events

**Output:** Pass/Fail + failure count

**Best Practice:**
- ✅ Contracts are domain-specific (earthquake physics, not just "data exists")
- ✅ Fail fast: Pipeline should halt if critical contracts fail
- ✅ Contracts prevent silent data corruption

#### **4.2 Anomaly Detection (Soft Flags)**
Flag unusual patterns for investigation (not necessarily errors):
- `anom_mag_missing` - Missing magnitude
- `anom_mag_extreme` - Magnitude < 0 or > 8.0
- `anom_depth_negative` - Depth < 0 (above sea level)
- `anom_depth_extreme` - Depth > 500 km (very deep subduction)
- `anom_coords_missing` - Missing location

**Output:** Investigation table showing flagged events

**Best Practice:**
- ✅ Anomalies ≠ Errors (deep earthquakes are real, just unusual)
- ✅ Create `anom_any` summary flag for quick filtering
- ✅ Display sample anomalous records for manual review

#### **4.3 Canaries (Distribution Checks)**
Simple aggregate metrics to detect data drift:
- Daily event counts: `min`, `median`, `max`, `n_days`
- Spike detection: Days with > 2.5× median events

**Output:** Dictionary of metrics + list of spike days

**Best Practice:**
- ✅ Canaries are cheap, high-signal metrics
- ✅ Compare across runs to detect: source changes, API issues, seasonal patterns
- ✅ Example: If median magnitude suddenly shifts, investigate data source

---

### **Stage 5: Leakage Audit**
**Objective:** Prevent future data from contaminating training

**Key Questions:**
1. Are preprocessing stats computed on TRAIN only? (not entire dataset)
2. Are joins pulling in data that arrives after event time?
3. Is `updated_utc` used as a feature? (it gets revised later!)
4. Are group summaries using future rows? (e.g., "this month's average" includes future)
5. Is prediction time clearly defined?

**Best Practice:**
- ✅ Leakage breaks time-series models silently
- ✅ Review checklist before feature engineering
- ✅ Example: Don't use aftershock counts to predict earthquake magnitude (aftershocks happen AFTER)

**Common Leaky Features in Earthquake Data:**
- ❌ `updated_utc` - Gets revised after event
- ❌ `status` - Changes from "automatic" → "reviewed" later
- ❌ Aftershock counts
- ✅ Historical summaries ("past 30 days in this region") are safe

---

### **Stage 6: Write Validation Report + Run Log**

#### **Validation Report**
Comprehensive JSON capturing:
- Contract results (pass/fail, details)
- Anomaly statistics (counts by type)
- Canary metrics (distribution stats)
- Leakage checklist

**File:** `data/reference/validation_report_{RUN_ID}.json`

**Use Case:** Compare reports across runs to detect drift

#### **Pipeline Run Log**
Links inputs → outputs for full traceability:
- Input fingerprint (query parameters hash)
- All input file paths and sizes
- All output file paths
- Row definition
- Operational notes

**File:** `data/reference/pipeline_runs/run_{RUN_ID}.json`

**Use Case:** "This curated file came from which raw data?"

**Best Practice:**
- ✅ Run logs enable reproducibility
- ✅ Fingerprints detect when same query produces different results (API changes)
- ✅ Each run is self-documenting

---

## Key Design Patterns

### **1. ELT (Extract-Load-Transform) Architecture**
```
Raw (load as-is) → Staged (types/structure) → Curated (features)
```
- Preserves raw data for reprocessing
- Staged layer is the "clean checkpoint"
- Curated layer is analysis-specific

### **2. Run ID Consistency**
All artifacts from one execution share the same `RUN_ID`:
```
usgs_pages_run_20260203_152345_utc.jsonl
usgs_earthquakes_staged_run_20260203_152345_utc.parquet
usgs_earthquakes_curated_run_20260203_152345_utc.parquet
validation_report_20260203_152345_utc.json
run_20260203_152345_utc.json
```
- Enables tracing: "Which raw data produced this curated file?"
- Enables comparison: "How did validation change between runs?"

### **3. Separation of Concerns**
- **Staging** = Data quality (types, dedup, normalization)
- **Curating** = Feature engineering (derived columns, binning)
- **Validation** = Quality assurance (contracts, anomalies, drift)

Each stage has a single responsibility.

### **4. Metadata First**
Every file has accompanying metadata:
- Raw → Query parameters, fetch stats
- Staged → Schema, quality metrics, deduplication
- Curated → Feature definitions, partitioning
- Validation → Contract results, anomaly counts, canaries
- Run Log → Input/output traceability

---

## What Makes This a Good ML Pipeline Example?

### ✅ **Production-Ready Practices**
1. **Error handling** - Pagination handles API limits, type coercion handles bad data
2. **Validation** - Contracts catch data quality issues early
3. **Reproducibility** - Run IDs link artifacts, metadata documents decisions
4. **Scalability** - Partitioning enables efficient querying on large datasets
5. **Maintainability** - Clear stage separation makes debugging easier

### ✅ **Educational Value**
1. **Real-world data** - USGS API has messy, nested JSON (not clean CSVs)
2. **Complete workflow** - Ingestion → Staging → Curation → Validation
3. **Best practices** - Demonstrates contracts, anomaly detection, leakage prevention
4. **Documented decisions** - Comments explain "why", not just "what"

### ✅ **Extensibility**
1. **New features?** Add to curated stage without re-fetching raw data
2. **Different time window?** Change query parameters, run pipeline again
3. **Additional validation?** Add contracts without changing staging logic
4. **ML modeling?** Curated data is ready, validation report shows data quality

---

## Common Pitfalls This Pipeline Avoids

❌ **Overwriting raw data** → ✅ Each run creates timestamped files
❌ **Mixing staging and features** → ✅ Separate stages with clear responsibilities
❌ **No validation** → ✅ Contracts + anomalies + canaries
❌ **Manual tracking** → ✅ Automated run logs and metadata
❌ **Data leakage** → ✅ Leakage audit checklist
❌ **Losing provenance** → ✅ Run ID links all artifacts
❌ **Silent failures** → ✅ Contracts fail loudly with details

---

## How to Use This as a Template

### For Similar API-Based Pipelines:
1. **Stage 1 (Ingest)** - Replace USGS API with your source, keep pagination pattern
2. **Stage 2 (Stage)** - Adjust `staged_dataframe()` function for your schema
3. **Stage 3 (Curate)** - Define features relevant to your domain
4. **Stage 4 (Validate)** - Define contracts based on your domain logic
5. **Stages 5-6** - Leakage audit and logging patterns remain the same

### For CSV/File-Based Pipelines:
- Skip pagination logic in Stage 1
- Keep validation, staging, and curation patterns identical

### For Different Domains:
- **E-commerce:** Events = purchases, features = customer segments, contracts = valid prices
- **IoT sensors:** Events = readings, features = rolling windows, contracts = sensor ranges
- **Finance:** Events = transactions, features = time-based aggregations, contracts = balance checks

---

## Next Steps: From Pipeline to ML

This pipeline produces analysis-ready data. To add ML modeling:

1. **Train/Test Split**
   - Use `event_day` for time-based splits (train on Jan 1-20, test on Jan 21-30)
   - Never shuffle time-series data randomly

2. **Feature Engineering**
   - Historical features: "Earthquakes in past 7 days in 100km radius"
   - Geographic features: "Distance to nearest fault line"
   - Remember: Compute stats on train only, apply to test

3. **Model Training**
   - Use curated parquet files (fast to load)
   - Check validation report before training (data quality OK?)
   - Example tasks: Predict aftershock likelihood, classify event type

4. **Model Validation**
   - Use same contract validation on predictions
   - Check for distribution drift (canaries on prediction outputs)

---

## File Structure Summary

```
work/m2_project/
├── data/
│   ├── raw/
│   │   ├── usgs_pages_run_{RUN_ID}.jsonl          # API responses
│   │   └── usgs_meta_run_{RUN_ID}.json            # Query metadata
│   ├── staged/
│   │   ├── usgs_earthquakes_staged_{RUN_ID}.parquet
│   │   └── usgs_staged_meta_{RUN_ID}.json
│   ├── warehouse/
│   │   ├── usgs_earthquakes_curated_{RUN_ID}.parquet
│   │   └── partitions/
│   │       ├── event_day=2026-01-01/earthquakes.parquet
│   │       ├── event_day=2026-01-02/earthquakes.parquet
│   │       └── ...
│   └── reference/
│       ├── validation_report_{RUN_ID}.json
│       └── pipeline_runs/
│           └── run_{RUN_ID}.json
```

---

## Conclusion

This notebook demonstrates a **mini ML pipeline** that:
- ✅ Handles real-world messy data (nested JSON, pagination, missing values)
- ✅ Follows data engineering best practices (ELT, validation, provenance)
- ✅ Prevents common pitfalls (leakage, silent failures, lost metadata)
- ✅ Produces analysis-ready, partitioned data with quality guarantees

**Use this as a template** for building robust data pipelines in any domain. The patterns (staging, contracts, run logs, leakage audits) are universal, even when the data source and features change.